In [ ]:
!pip install google-cloud-tasks==2.2.0

In [ ]:
!gcloud config list

In [ ]:
import json
import csv
import argparse
import subprocess
import urllib.parse
from google.cloud import tasks_v2

In [ ]:
# projet GCP
result = subprocess.run(['gcloud', 'config', 'get-value', 'project'], stdout=subprocess.PIPE)
project = result.stdout.decode('utf-8')[0:-1]
#print(project)
# liste des regions
result = subprocess.run(['cat', './regions-list.txt'], stdout=subprocess.PIPE)
regions = result.stdout.decode('utf-8').split('\n')
regions.pop()
#print(regions)
# liste des urls Cloud Run
result = subprocess.run(['cat', './urls-list.txt'], stdout=subprocess.PIPE)
urls = result.stdout.decode('utf-8').split('\n')
urls.pop()
for i in range(0,9):
    urls[i] = urls[i] + "/stroccurences"
    #print(urls)
    # list des tasks queues
    queues = []
    for r in regions:
        q = "wsreqsqueue-" + r
        queues.append(q)
        #print(queues)

In [ ]:
# Create the clients
clients = []
for i in range(0,9):
    clients.append(tasks_v2.CloudTasksClient())
    # Construct the fully qualified queue names
parents = []
for i in range(0,9):
    parents.append(clients[i].queue_path(project, 'europe-west1', queues[i]))
print(parents)

In [ ]:
def build_request(url, payload):
    task = {
        "view": tasks_v2.types.Task.View.FULL,
        "http_request": {  # Specify the type of request.
             #"http_method": tasks_v2.types.HttpMethod.POST,
             "url": url,  # The full url path that the task will be sent to.
             "oidc_token": {"service_account_email": "my-test-sa@pascal-sandbox.iam.gserviceaccount.com"},
         }
    }

    if payload is not None:
        if isinstance(payload, dict):
            # Convert dict to JSON string
            payload = json.dumps(payload)
            # specify http content-type to application/json
            task["http_request"]["headers"] = {"Content-type": "application/json"}

        # The API expects a payload of type bytes.
        converted_payload = payload.encode()

        # Add the payload to the request.
        task["http_request"]["body"] = converted_payload

    return task

In [ ]:
def process_requests_file(tasks_client, parent, reqs_filename, run_url):
    print("Processing file: " + reqs_filename + " to queue: " + parent + " for run to service: " + run_url + "\n")
    with open(reqs_filename, newline='') as infile:
        rowreader = csv.reader(infile, delimiter=';')
        nb = 1
        for row in rowreader:
            if row[0]:
                if row[1]:
                    search = row[0] + " " + row[1]
                else:
                    search = row[0]
            else:
                search = row[1]
            #print(search)
            task = build_request(run_url, {'query': search, 'lexique': row[0], 'categorie': row[1]})
            #print("creating task for string " + search + " = lex: " + row[0] + " + cat: " + row[1])
            response = tasks_client.create_task(request={"parent": parent, "task": task})
            #print("Created task {}".format(response.name) +  "for string " + search)
            if nb % 1000 == 0:
                print(reqs_filename + " : " + str(nb))
                #print(search)
                #print(task)
            nb = nb + 1
    return "file {} processed".format(reqs_filename)

In [ ]:
region = 0
process_requests_file(clients[region], parents[region], 'xaa', urls[region])